In [65]:
import ee
import geemap
import geopandas as gpd
import json

# ee.Authenticate()
ee.Initialize(project="ee-zuzannaslobodzian2")

In [66]:
with open("C:\\space_technologies\\OpenPitMining\\niedzwedzia_gora\\niedzwiedzia_gora_polygon.geojson") as f:
    geojson_data = json.load(f)

polygon = geemap.geojson_to_ee(geojson_data)

image = (
    ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
    .filterBounds(polygon.geometry())
    .filterDate('2025-03-01', '2025-03-31')
    .sort('CLOUDY_PIXEL_PERCENTAGE')
    .first()
)

In [67]:
rectangle = polygon.geometry().bounds().buffer(500)

image = image.clip(rectangle)

rgb = image.select(['B4', 'B3', 'B2'])
ndvi = image.normalizedDifference(['B8', 'B4'])
bare_soil_mask = ndvi.lt(0.18)

ndvi_vis = {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}
rgb_vis = {'min': 0, 'max': 3000}
polygon_style = {
    "color": "red",
    "fillColor": "00000000",
    "width": 2
}

map = geemap.Map()
map.centerObject(polygon, zoom=15)
map.add_layer(rgb, rgb_vis, "ortofotomapa")
map.add_layer(ndvi, ndvi_vis, "NDVI")
map.add_layer(polygon.style(**polygon_style), {}, "obszar górniczy")
map


Map(center=[50.10339412565508, 19.63649860680545], controls=(WidgetControl(options=['position', 'transparent_b…

In [68]:
bare_soil_mask = ndvi.lt(0.18)


bare_soil_vis = {'min': 0, 'max': 1, 'palette': ['black', 'white']}
mask_vis = {'min': 0, 'max': 1, 'palette': ['white', 'yellow']}


map = geemap.Map()
map.centerObject(polygon, zoom=15)
map.add_layer(rgb, rgb_vis, "ortofotomapa")
map.add_layer(ndvi, ndvi_vis, "NDVI")

map.add_layer(bare_soil_mask, bare_soil_vis, "Bare Soil Mask")

map.add_layer(polygon.style(**polygon_style), {}, "obszar górniczy")
map

Map(center=[50.10339412565508, 19.63649860680545], controls=(WidgetControl(options=['position', 'transparent_b…